In [ ]:
# !pip install langchain
# !pip install langchain-openai

# Introduction to LangChain 

Working with LLMs involves in one way or another working with a specific type of abstraction: "Prompts".

However, in the practical context of day-to-day tasks we expect LLMs to perform, these prompts won't be some static and dead type of abstraction. Instead we'll work with dynamic prompts re-usable prompts.

# Lanchain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework that allows you to connect LLMs together by allowing you to work with modular components like prompt templates and chains giving you immense flexibility in creating tailored solutions powered by the capabilities of large language models.


Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

LangChain facilitates the creation of complex pipelines that leverage the connection of components like chains, prompt templates, output parsers and others to compose intricate pipelines that give you everything you need to solve a wide variety of tasks.

At the core of LangChain, we have the following elements:

- Models
- Prompts
- Output parsers

**Models**

Models are nothing more than abstractions over the LLM APIs like the ChatGPT API.​

In [2]:
# Uncomment and add your api key!!!


# import os


# os.environ["OPENAI_API_KEY"] = "<your OPENAI API KEY>"

In [10]:
# !pip install langchain-openai

In [1]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125")

output = chat_model.invoke("I am teaching a live-training\
    about LLMs!")

print(output.content)

That's great! LLM stands for Master of Laws, and it's a postgraduate degree for lawyers or law graduates looking to specialize in a specific area of law or gain expertise in a particular legal field. 

During your live-training, you can cover topics such as the benefits of pursuing an LLM, the different types of LLM programs available, admission requirements, and potential career opportunities after completing an LLM. Additionally, you can discuss the importance of networking, internships, and research opportunities for LLM students.

It's also important to address any common misconceptions about LLM programs and provide tips for prospective students on how to choose the right program for their career goals. Overall, your live-training can be a valuable resource for anyone considering pursuing an LLM degree. Good luck with your training!


In [3]:
from langchain.chat_models import ChatOllama

model = ChatOllama(model="mistral", format_chat=True)

model.invoke("Hi!")

AIMessage(content='Hello! How can I help you today?')

You can predict outputs from both LLMs and ChatModels:

Basic components are:

- Models
- Prompt templates
- Output parsers

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Show me 5 examples\
    of this concept: {concept}")

prompt.format(concept="animal")

'Human: Show me 5 examples    of this concept: animal'

In [21]:
chain = prompt | chat_model

output = chain.invoke({"concept": "animal"})
output

AIMessage(content='1. Dogs are a popular choice for a pet animal due to their loyalty and companionship.\n2. Elephants are known for their intelligence and strong social bonds within their herds.\n3. Dolphins are highly intelligent animals that are known for their playful behavior and complex communication skills.\n4. Giraffes are the tallest animals on land and have long necks that help them reach leaves high up in trees.\n5. Penguins are flightless birds that are well-adapted to living in cold climates and rely on teamwork to survive.')

In [22]:
from IPython.display import Markdown

Markdown(output.content)

1. Dogs are a popular choice for a pet animal due to their loyalty and companionship.
2. Elephants are known for their intelligence and strong social bonds within their herds.
3. Dolphins are highly intelligent animals that are known for their playful behavior and complex communication skills.
4. Giraffes are the tallest animals on land and have long necks that help them reach leaves high up in trees.
5. Penguins are flightless birds that are well-adapted to living in cold climates and rely on teamwork to survive.

You can also use the predict method over a string input:

In [24]:
text = "What would be a good name for a dog that loves to nap??"
chat_model.invoke(text)

AIMessage(content='Snooze')

**Prompts**

The same works for prompts. Now, prompts are pieces of text we feed to LLMs, and LangChain allows you to work with prompt templates.

Prompt Templates are useful abstractions for reusing prompts and they are used to provide context for the specific task that the language model needs to complete. 

A simple example is a `PromptTemplate` that formats a string into a prompt:

In [34]:
from langchain_core.prompts  import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="sleeping")
# Output: "What is a good dog name for a dog that loves to nap?"

'Human: What is a good dog name for a dog that loves to sleeping?'

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [24]:
chain.invoke({"concept": "Landscapes"})

AIMessage(content='1. A serene countryside landscape with rolling hills, colorful wildflowers, and a clear blue sky.\n2. A rugged mountain landscape with towering peaks, snow-capped summits, and a winding river below.\n3. A tropical beach landscape with palm trees swaying in the breeze, white sandy shores, and crystal clear waters.\n4. A desert landscape with vast stretches of sand dunes, sparse vegetation, and a blazing sun overhead.\n5. A lush forest landscape with towering trees, a carpet of ferns, and a tranquil stream meandering through the woods.')

In [29]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [31]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_template("""
Write 5 concepts that are fundamental to learn about {topic}.
                                          """)
chain = prompt | llm | output_parser
chain.invoke({"topic": "Artificial Neural Networks"})

This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

Now, to put everything together LangChain allows you to build something called "chains", which are components that connect prompts, llms and output parsers into a building block that allows you to create more interesting and complex functionality.

Let's look at the example below:

In [36]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")

chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
)
chain.invoke("sleep")

{'activity': 'sleep', 'text': 'Snooze'}

So, what the chain is doing is connecting these basic components (the LLM and the prompt template) into
a block that can be run separately. The chain allows you to turn workflows using LLLMs into this modular process of composing components.

Now, the newer versions of LangChain have a new representation language to create these chains (and more) known as LCEL or LangChain expression language, which is a declarative way to easily compose chains together. The same example as above expressed in this LCEL format would be:

In [37]:
chain = prompt | ChatOpenAI()

chain.invoke({"activity": "sleep"})

AIMessage(content='Snooze')

Notice that now the output is an `AIMessage()` object, which represents LangChain's way to abstract the output from an LLM model like ChatGPT or others.

These building blocks and abstractions that LangChain provides are what makes this library so unique, because it gives you the tools you didn't know you need it to build awesome stuff powered by LLMs.

# LangChain Exercise

Let's create a simple chain for summarization of content. 

Your chain should:

- A prompt template with one or more variables
- A model like ChatGPT or other (you can use local models if you'd like, I recommend `ChatOllama` for that!)
- Optional: use output parsing or just fetch the string output at the end!

## Example Answer

Let's make use of the `ChatPromptTemplate` to abstract away the following pieces of the prompt: 
- `content` - the text content to be summarized  
- `summary_format` - the format in which we want the summary to be presented (like bullet points and so on).

In [29]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Summarize this: {content}. The output should be in the following format: {summary_format}.")

# We can look at a simple example to illustrate what that prompt is doing
prompt.format(content="This is a test.", summary_format="One word summary")

'Human: Summarize this: This is a test.. The output should be in the following format: One word summary.'

Ok, now that we have our prompt template done, let's load the llm and create a nice chain to put everything together. 

In [30]:
from langchain_openai import ChatOpenAI

llm_chat =  ChatOpenAI()
chain = prompt | llm_chat # This is the Pipe symbol! from LCEL that connect model to prompt!

Now, that we have our chain we can run some tests. The cool thing about working with LLMs is that you can use them to create examples for simple tests like this (avoiding the annoynace of searching online for some piece of text, copying and pasting etc...). So, let's generate a few examples of tests below:

In [31]:
num_examples = 3
examples = []
for i in range(num_examples):
    examples.append(llm_chat.invoke("Create a piece of text with 2 paragraphs about a random topic regarding human-machine interaction."))

examples

[AIMessage(content='Human-machine interaction has become an integral part of daily life, with technology playing a crucial role in various aspects of society. From smartphones to smart homes, humans now rely on machines for communication, work, entertainment, and even health care. As these technologies continue to advance, the relationship between humans and machines is becoming increasingly complex.\n\nOne of the key challenges in human-machine interaction is ensuring that machines are designed in a way that is intuitive and user-friendly. Users should be able to easily navigate through interfaces and interact with machines without feeling overwhelmed or confused. Additionally, there is a growing concern about the ethical implications of human-machine interaction, particularly in areas such as artificial intelligence and automation. As machines become more autonomous and intelligent, questions arise about accountability, privacy, and the impact on job markets. Finding a balance betwee

Nice! Now that we have our examples, let's run our chain on them and check out the results.

In [32]:
summary_format = "bullet points"

outputs = []
for ex in examples:
    outputs.append(chain.invoke({"content": ex, "summary_format": summary_format}))

# Let's display one example output
outputs[0]

AIMessage(content='- Human-machine interaction is an integral part of daily life\n- Technology plays a crucial role in various aspects of society\n- Humans rely on machines for communication, work, entertainment, and health care\n- Relationship between humans and machines is becoming increasingly complex\n- Key challenge is ensuring machines are designed in an intuitive and user-friendly way\n- Growing concern about ethical implications of human-machine interaction\n- Questions about accountability, privacy, and impact on job markets\n- Finding a balance between benefits of technological advancements and potential risks is essential')

Great! So it seems our chain worked and we generated some summaries! Let's visualize all the summaries generated in a neat way.

In [33]:
from IPython.display import Markdown

for i in range(num_examples):
    display(Markdown(f"Output {i} \n {outputs[i].content}"))
# Markdown(f"**Input**: {examples[0]}\n\n**Output**: {outputs[0]}")

Output 0 
 - Human-machine interaction is an integral part of daily life
- Technology plays a crucial role in various aspects of society
- Humans rely on machines for communication, work, entertainment, and health care
- Relationship between humans and machines is becoming increasingly complex
- Key challenge is ensuring machines are designed in an intuitive and user-friendly way
- Growing concern about ethical implications of human-machine interaction
- Questions about accountability, privacy, and impact on job markets
- Finding a balance between benefits of technological advancements and potential risks is essential

Output 1 
 - Artificial intelligence (AI) is a fascinating aspect of human-machine interaction
- AI has the ability to learn and adapt to new information, making it valuable in various industries
- AI is revolutionizing the way we interact with machines, from self-driving cars to virtual assistants
- Ethical questions arise about integrating AI into daily lives and ensuring machines act in humanity's best interest
- Haptic feedback technology allows users to feel physical sensations when interacting with machines
- Haptic feedback is commonly used in virtual reality systems to create a more immersive experience
- Haptic feedback has the potential to revolutionize industries like gaming, healthcare, and manufacturing
- As technology continues to advance, the possibilities for human-machine interaction are endless and exciting

Output 2 
 - AI is increasingly integrated into daily routines through virtual assistants and social media algorithms
- Raises questions about impact on human autonomy and decision-making
- Ethical implications of relying on machines for decision-making need careful consideration
- Technology has the potential to enhance abilities and improve quality of life
- Examples include prosthetic limbs controlled by neural signals and exoskeletons for individuals with disabilities
- It is essential to ensure technology advancements are accessible and inclusive for all individuals
- Embracing human-machine interaction can create a more inclusive and empowered society.

Great! Our summaries worked, and we were able to apply a given summary format to all of them.

LangChain is an extremely powerful library to work with abstractions like these and throughout this course we hope to give you a gliimpse of the cool stuff you can build with it.